In [1]:
import uuid
from pydantic import BaseModel
from typing import List, Dict, Optional
from datetime import datetime

In [2]:
class UserDetails(BaseModel):
    UserName: Optional[str]
    UserID: str
    DateCreated: Optional[str]
    Preferences: Dict
    Email: Optional[str]

In [3]:
class StoreMemory():
    def __init__(self):
        self.users = {}
        self.trip_plans = {}
    def add_or_get_user(self, user_id: str = None, user_name = None, preferences = None, email: str = None):
        if preferences is None:
            preferences = {}
        if  user_id in self.users:
            return self.users[user_id]
        else:
            now = datetime.now().isoformat()
            user_details = UserDetails(
                UserName = user_name,
                UserID = user_id,
                DateCreated = now,
                Preferences = preferences,
                Email = email
            )
            self.users[user_id] = user_details
            return user_details

    def add_travel_details(self, user_id, Departure: str, Destination: List[str], StartDate: str, Duration: str, Budget: str, Interest: List[str], ExtraDetail: List[str], AllDetails: bool):
        trip_id = str(uuid.uuid4())
        trip_id = str(Destination) + trip_id
        trip_info = {
            "user_id": user_id,
            "trip_id": trip_id,
            "Departure": Departure,
            "Destination":Destination,
            "StartDate": StartDate,
            "Duration": Duration, 
            "Budget": Budget,
            "Interest": Interest,
            "ExtraDetail": ExtraDetail,
            "AllDetails": AllDetails
        }
        self.trip_plans[trip_id] = trip_info

    def get_trip_details(self, user_id, specific_trip = False):
        if specific_trip == False:
            trip_details = [trips  for trips in self.trip_plans.values() if trips["user_id"] == user_id]
            return trip_details


In [4]:
def test_store_memory():
    store = StoreMemory()

    # ---- Test user creation ----
    user = store.add_or_get_user(
        user_id="u123",
        user_name="Ahmed",
        preferences={"hotel": "luxury"},
        email="ahmed@test.com"
    )

    print("User created:", user)

    # ---- Test getting same user again ----
    same_user = store.add_or_get_user(user_id="u123")
    print("Same user fetched:", same_user)

    assert user is same_user

    # ---- Add trip details ----
    store.add_travel_details(
        user_id="u123",
        Departure="Islamabad",
        Destination=["Tokyo", "Kyoto"],
        StartDate="2026-03-01",
        Duration="7 days",
        Budget="3000 USD",
        Interest=["food", "culture"],
        ExtraDetail=["cherry blossom season"],
        AllDetails=True
    )

    # ---- Fetch trips ----
    trips = store.get_trip_details(user_id="u123")
    print("Trips found:", trips)

    assert len(trips) == 1
    assert trips[0]["Departure"] == "Islamabad"
    assert "Tokyo" in trips[0]["Destination"]

    print("✅ All tests passed!")


test_store_memory()


User created: UserName='Ahmed' UserID='u123' DateCreated='2026-01-12T09:40:45.639260' Preferences={'hotel': 'luxury'} Email='ahmed@test.com'
Same user fetched: UserName='Ahmed' UserID='u123' DateCreated='2026-01-12T09:40:45.639260' Preferences={'hotel': 'luxury'} Email='ahmed@test.com'
Trips found: [{'user_id': 'u123', 'trip_id': "['Tokyo', 'Kyoto']7517687b-a12a-43c6-98b8-61d1bc47f9d5", 'Departure': 'Islamabad', 'Destination': ['Tokyo', 'Kyoto'], 'StartDate': '2026-03-01', 'Duration': '7 days', 'Budget': '3000 USD', 'Interest': ['food', 'culture'], 'ExtraDetail': ['cherry blossom season'], 'AllDetails': True}]
✅ All tests passed!
